# Chapter two
collecting weather broadcast.
with the requests library we can get the daily weather broadcast with only a few lines of code. there are many free api providers but for this project i choosed to use openweathermap.
with this function above we can give an array [] with city names as argument, looping over the cities and in the end concatinate it as result to a dataframe with every information that we choose to collect.





***KEEP IN MIND TO KEEP YOUR API KEY SECURE ***


In [ ]:
import pandas as pd
import requests
# GET THE KEY
import os
from dotenv import load_dotenv
load_dotenv()
UR_API_KEY = os.getenv('DB_WEATHER_KEY')

def weather_boradcast(towns_list):
  dataframes_for_each_town = []
  API_key = UR_API_KEY
  for town in towns_list:
    weather = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={town}&appid={API_key}&units=metric")
    weather_resp = weather.json()
    # normalize method can create the dataframe immidiately from an json/ dictionary 
    weather_df= pd.json_normalize(
    weather_resp['list'],
    record_path=['weather'], 
    meta=['dt_txt', ['main', 'temp'], ['main', 'feels_like'], ['clouds', 'all'], ['rain', '3h'], ['snow', '3h'], ['wind', 'speed'], 
          ['wind', 'deg'], ['main', 'humidity'], ['main', 'pressure']],
    errors='ignore'
    )
    weather_df.drop(['wind.deg', 'main.humidity', 'main.pressure','main.feels_like','icon','wind.deg',"rain.3h","clouds.all",	"snow.3h" ], axis = 1, inplace = True) 
    weather_df.rename(columns={'main': 'outlook',
                             'description': 'detailed_outlook',
                             'dt_txt': 'forecast_time',
                             'main.temp': 'temperature',
                             
                             'wind.speed': 'wind_speed',
                             },
                    inplace=True)

    weather_df.insert(0, 'city', weather_resp["city"]["name"])
    weather_df.insert(1, 'country', weather_resp["city"]["country"])
    
    dataframes_for_each_town.append(weather_df)

  return pd.concat(dataframes_for_each_town, ignore_index=True)


And here is the second dataframe

In [ ]:
top_cities_weather= weather_boradcast([ "Madrid","Rome","Paris"])
top_cities_weather.sample(10)

,city,country,id,outlook,detailed_outlook,forecast_time,temperature,wind_speed
57,Rome,US,804,Clouds,overcast clouds,2022-12-02 21:00:00,14.95,4.49
72,Rome,US,804,Clouds,overcast clouds,2022-12-04 18:00:00,14.12,0.9
99,Paris,FR,804,Clouds,overcast clouds,2022-12-03 03:00:00,3.99,3.38
21,Madrid,ES,803,Clouds,broken clouds,2022-12-03 09:00:00,6.74,0.42
113,Paris,FR,801,Clouds,few clouds,2022-12-04 21:00:00,3.56,2.96
63,Rome,US,501,Rain,moderate rain,2022-12-03 15:00:00,14.15,0.18
87,Paris,FR,800,Clear,clear sky,2022-12-01 15:00:00,7.81,3.39
100,Paris,FR,804,Clouds,overcast clouds,2022-12-03 06:00:00,3.76,3.23
0,Madrid,ES,802,Clouds,scattered clouds,2022-11-30 18:00:00,10.37,0.85
40,Rome,US,803,Clouds,broken clouds,2022-11-30 18:00:00,15,5.37
